# Check for suppression of X-rays

For this, the reference detectors were removed from the lead container and afterwards placed inside the lead container.

Measurement took place on 2020-01-30:

- /Users/hkromer/02_PhD/02_Data/13.new_chamber/2020-01-30.EmittingSpotSize_test.xlsx
- /Users/hkromer/02_PhD/02_Data/13.new_chamber/2020-01-30.EmittingSpotSize_test_lead_around_detectors.xlsx

# Load data from Database

In [ ]:
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy as sql
import matplotlib.pyplot as plt
import matplotlib.dates as md
import seaborn as sns
sns.set()
import getopt
import sys
import datetime
from scipy.interpolate import interp1d
from NGDataObject import NGDataObject

#*******************************
# CONSTANTS
#*******************************
# connection to database
CREDENTIALS_FILE = '/Users/hkromer/02_PhD/01.github/dash_NG/credentials.pw'
DB = "NG_twofast_DB" # name of the database
HOST = "twofast-RPi3-0"  # database host

# LOOKUP TABLES
LUT_PRESSURE_ION_SOURCE = "/Users/hkromer/02_PhD/01.github/phd/01_neutron_generator_contol/LUT_pressure_ion_source.txt"

In [ ]:
DAY = "2020-01-30" # day to extract from the db

In [ ]:
# read password and user to connect to database
credentials = pd.read_csv(CREDENTIALS_FILE, header=0)
user = credentials['username'].values[0]
pw = credentials['password'].values[0]


#*******************************
# GET DATA
#*******************************

# connect to DB
con = NGDataObject(host = HOST, database=DB, user=user, password=pw)

# get dose
query = "SELECT * FROM data_dose WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_dose = con.get_from_database(query=query)

# get HV
query = "SELECT * FROM data_HV WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_hv = con.get_from_database(query=query)
data_hv['HV_current_x100'] = data_hv['HV_current']*100.0

# get pressure
query = "SELECT * FROM data_pressure WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_pressure = con.get_from_database(query=query)

# get reference detectors
query = "SELECT * FROM data_referenceDetectors WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_reference_full = con.get_from_database(query=query)

# get microwave power
query = "SELECT * FROM microwave_generator_power WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_microwave_power_full = con.get_from_database(query=query)

# get microwave frequency
query = "SELECT * FROM microwave_generator_frequency WHERE DATE(time) = '%(t)s'" % {"t": DAY}
data_microwave_frequency_full = con.get_from_database(query=query)

In [ ]:
# save to file
data_reference_full['sum_counts'] = data_reference_full['counts_D1'] + data_reference_full['counts_D2'] + data_reference_full['counts_D3'] + data_reference_full['counts_D4'] 

path = '/Users/hkromer/02_PhD/02_Data/13.new_chamber/DATA/x_ray_suppression/'
data_dose.to_csv(f'{path}/2020-02-06_dose.csv')
data_hv.to_csv(f'{path}/2020-02-06_hv.csv')
data_pressure.to_csv(f'{path}/2020-02-06_pressure.csv')
data_reference_full.to_csv(f'{path}/2020-02-06_reference_detectors.csv')
data_microwave_power_full.to_csv(f'{path}/2020-02-06_microwave_power.csv')
data_microwave_frequency_full.to_csv(f'{path}/2020-02-06_microwave_frequency.csv')

# Influence on the microwave on the count rate in the detector

## Microwave on: 13:32 until 14:30
## Microwave off: 16:30 until 17:20

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_reference_full.index, y='sum_counts', data=data_reference_full, ax=ax)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M:%S'))
# ax.set(yscale="log")
ax.set_ylabel('counts reference detector [-]')
ax.set_xlabel(f'Time, {DAY}')

In [ ]:
data_microwave_power_full.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_microwave_power_full.index, y='FP', data=data_microwave_power_full, ax=ax)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M:%S'))
# ax.set(yscale="log")
ax.set_ylabel('Forward power [W]')
ax.set_xlabel(f'Time, {DAY}')

In [ ]:
start_mw = '2020-01-30 13:30:00'
end_mw = '2020-01-30 14:30:00'
data_reference_s = data_reference_full.loc[start_mw:end_mw, :]

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_reference_s.index, y='sum_counts', data=data_reference_s, ax=ax)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M:%S'))
# ax.set(yscale="log")
ax.set_ylabel('counts reference detector [-]')
ax.set_xlabel(f'Time, {DAY}')

In [ ]:
data_microwave_power_s = data_microwave_power_full.loc[start_mw:end_mw, :]
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_microwave_power_s.index, y='FP', data=data_microwave_power_s, ax=ax)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M:%S'))
# ax.set(yscale="log")
ax.set_ylabel('Forward power [W]')
ax.set_xlabel(f'Time, {DAY}')

In [ ]:
start_no_mw = '2020-01-30 16:33:00'
end_no_mw = '2020-01-30 17:20:00'
data_reference_s = data_reference_full.loc[start_no_mw:end_no_mw, :]

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_reference_s.index, y='sum_counts', data=data_reference_s, ax=ax)
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M:%S'))
# ax.set(yscale="log")
ax.set_ylabel('counts reference detector [-]')
ax.set_xlabel(f'Time, {DAY}')

In [ ]:
data_microwave_power_full.loc[start_no_mw:end_no_mw, :]


In [ ]:
start_mw = '2020-01-30 13:30:00'
end_mw = '2020-01-30 14:30:00'

start_no_mw = '2020-01-30 16:33:00'
end_no_mw = '2020-01-30 17:20:00'

# counts when on and off
ref_counts_on = data_reference_full.loc[start_mw:end_mw, :]
ref_counts_off = data_reference_full.loc[start_no_mw:end_no_mw, :]

ref_counts_on = ref_counts_on.reset_index()
ref_counts_off = ref_counts_off.reset_index()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_on.index, y='sum_counts', data=ref_counts_on, ax=ax, color='darkred')
sns.lineplot(x=ref_counts_off.index, y='sum_counts', data=ref_counts_off, ax=ax, color='darkblue')

# ax.set(yscale="log")
ax.set_ylabel('counts reference detector [-]')
ax.set_xlabel(f'30 seconds readout interval')

In [ ]:
# Combine reference detector counts into one dataframe
ref_counts_on['label'] = 'On'
ref_counts_off['label'] = 'Off'

ref_counts_combined = ref_counts_on.append(ref_counts_off).reset_index()
ref_counts_combined

In [ ]:

fig, ax = plt.subplots(figsize=(12,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.boxplot(x='label', y='sum_counts', data=ref_counts_combined, ax=ax, width=0.25)


# plt.title('Reference detectors')
# ax.set(yscale="log")
ax.set_ylabel('Counts in reference \n detectors [1/(30s)]')
ax.set_xlabel(f'Microwave')

In [ ]:
print(f'Data points (30 sec intervals) when MW on: {ref_counts_on.shape[0]}, when MW off: {ref_counts_off.shape[0]}')
print(f'Mean ON, OFF: {ref_counts_on.sum_counts.mean()}, {ref_counts_off.sum_counts.mean()}')
print(f'Std ON, OFF: {ref_counts_on.sum_counts.std()}, {ref_counts_off.sum_counts.std()}')

In [ ]:
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

x_ecdf_mw_on, y_ecdf_mw_on = ecdf(ref_counts_on['sum_counts'])
x_ecdf_mw_off, y_ecdf_mw_off = ecdf(ref_counts_off['sum_counts'])

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=x_ecdf_mw_on, y=y_ecdf_mw_on, ax=ax, color='darkred', label='On')
sns.lineplot(x=x_ecdf_mw_off, y=y_ecdf_mw_off, ax=ax, color='darkblue', label='Off')

# ax.set(yscale="log")
ax.set_ylabel('ECDF [-]')
ax.set_xlabel(f'Counts')

# No lead around the detectors

# From 13:30 until 16:35


In [ ]:
def resample_30_seconds(df, cols, range_start, range_end):
    """
    Takes the a grouped df (grouped by day) and resamples the columns cols in 10s
    OUTPUT:
        - dataframe
    """

    d = {}

    s = pd.date_range(start=range_start, end=range_end, freq='30S')
    df_out = pd.DataFrame(pd.Series(s, name='time')).set_index('time')
    for col in cols:
        d[col] = df[col].resample('30S').mean()

    this_d = pd.DataFrame(d)

    df_out = df_out.merge(this_d, left_on=df_out.index, right_on=this_d.index, how='outer')
    df_out = df_out.set_index('key_0')
    
    return df_out

In [ ]:
start = '2020-01-30 14:35:00'
end = '2020-01-30 15:59:00'

### Plot HV, dose, microwave power, pressure, reference counts

In [ ]:
data_hv_s = data_hv.loc[start:end, :]
data_dose_s = data_dose.loc[start:end, :]
data_pressure_s = data_pressure.loc[start:end, :]
data_reference_s = data_reference_full.loc[start:end, :]
data_microwave_frequency_s = data_microwave_frequency_full.loc[start:end, :]
data_microwave_power_s = data_microwave_power_full.loc[start:end, :]

In [ ]:
data_dose_s.head()

In [ ]:
# remove outliers
data_dose_s = data_dose_s[data_dose_s['dose_corrected'] > 580]
data_reference_s = data_reference_s[data_reference_s['sum_counts'] > 3600]
data_hv_s = data_hv_s[data_hv_s['HV_voltage'] > 99.5]


# resample the 30 s
data_dose_s_rs = resample_30_seconds(data_dose_s, ['dose_corrected'], start, end)
data_dose_s_rs.index = data_dose_s_rs.index.rename('time')

data_reference_s_rs = resample_30_seconds(data_reference_s, ['sum_counts'], start, end)
data_reference_s_rs.index = data_reference_s_rs.index.rename('time')

data_hv_s_rs = resample_30_seconds(data_hv_s, ['HV_voltage', 'HV_current',], start, end)
data_hv_s_rs.index = data_hv_s_rs.index.rename('time')

In [ ]:
data_hv_s_rs

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv_s.index, y='HV_voltage', data=data_hv_s, ax=ax)
sns.lineplot(x=data_hv_s_rs.index, y='HV_voltage', data=data_hv_s_rs, ax=ax, color='darkred')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('HV [kV]')
ax.set_xlabel(f'Time, {DAY}')

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv_s.index, y='HV_current', data=data_hv_s, ax=ax)
sns.lineplot(x=data_hv_s_rs.index, y='HV_current', data=data_hv_s_rs, ax=ax, color='darkred')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('HV [mA]')
ax.set_xlabel(f'Time, {DAY}')

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_dose_s.index, y='dose_corrected', data=data_dose_s, ax=ax)
sns.lineplot(x=data_dose_s_rs.index, y='dose_corrected', data=data_dose_s_rs, ax=ax, color='darkred')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('dose [muSv/h]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,650)

In [ ]:
data_reference_s.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_reference_s.index, y='sum_counts', data=data_reference_s, ax=ax)
sns.lineplot(x=data_reference_s_rs.index, y='sum_counts', data=data_reference_s_rs, ax=ax, color='darkred')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts reference detector [1/(30s)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,4500)

## Normalize the reference counts by the dose

In [ ]:
ref_counts_norm_no_lead = data_reference_s_rs['sum_counts']/data_dose_s_rs['dose_corrected']
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_no_lead.index, y=ref_counts_norm_no_lead, ax=ax)

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by dose \n [1/(30s) * 1/(muSv/h)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,8)

## Normalize also by beam power

In [ ]:
data_hv_s_rs['beam_power'] = data_hv_s_rs['HV_voltage'] * data_hv_s_rs['HV_current']
ref_counts_norm_beam_no_lead = ref_counts_norm_no_lead/data_hv_s_rs['beam_power']

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_beam_no_lead.index, y=ref_counts_norm_beam_no_lead, ax=ax)

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by \n dose and beam power \n [1/(30s) * 1/(muSv/h) * 1/(W)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,0.125)

## Normalize by high voltage

In [ ]:
ref_counts_norm_HV_no_lead = ref_counts_norm_no_lead/data_hv_s_rs['HV_voltage']
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_HV.index, y=ref_counts_norm_HV, ax=ax)

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by \n dose and HV \n [1/(30s) * 1/(muSv/h) * 1/(Kv)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,0.1)

# Lead around the detectors

# From 17:54 until 19:30


In [ ]:
start = '2020-01-30 18:01:30'
end = '2020-01-30 19:29:00'

data_hv_s = data_hv.loc[start:end, :]
data_dose_s = data_dose.loc[start:end, :]
data_pressure_s = data_pressure.loc[start:end, :]
data_reference_s = data_reference_full.loc[start:end, :]
data_microwave_frequency_s = data_microwave_frequency_full.loc[start:end, :]
data_microwave_power_s = data_microwave_power_full.loc[start:end, :]

# remove outliers
data_dose_s = data_dose_s[data_dose_s['dose_corrected'] > 540]
data_reference_s = data_reference_s[data_reference_s['sum_counts'] > 3000]
data_hv_s = data_hv_s[data_hv_s['HV_current'] > 0.5]

# resample the 30 s
data_dose_s_rs = resample_30_seconds(data_dose_s, ['dose_corrected'], start, end)
data_dose_s_rs.index = data_dose_s_rs.index.rename('time')

data_reference_s_rs = resample_30_seconds(data_reference_s, ['sum_counts'], start, end)
data_reference_s_rs.index = data_reference_s_rs.index.rename('time')

data_hv_s_rs = resample_30_seconds(data_hv_s, ['HV_voltage', 'HV_current',], start, end)
data_hv_s_rs.index = data_hv_s_rs.index.rename('time')


fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_dose_s.index, y='dose_corrected', data=data_dose_s, ax=ax)
sns.lineplot(x=data_dose_s_rs.index, y='dose_corrected', data=data_dose_s_rs, ax=ax, color='darkred')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('dose [muSv/h]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,650)
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_reference_s.index, y='sum_counts', data=data_reference_s, ax=ax)
sns.lineplot(x=data_reference_s_rs.index, y='sum_counts', data=data_reference_s_rs, ax=ax, color='darkred')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts reference detector [1/(30s)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,4500)
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=data_hv_s.index, y='HV_current', data=data_hv_s, ax=ax)
sns.lineplot(x=data_hv_s_rs.index, y='HV_current', data=data_hv_s_rs, ax=ax, color='darkred')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('HV [mA]')
ax.set_xlabel(f'Time, {DAY}')

ref_counts_norm_lead = data_reference_s_rs['sum_counts']/data_dose_s_rs['dose_corrected']
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_lead.index, y=ref_counts_norm_lead, ax=ax)

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by dose \n [1/(30s) * 1/(muSv/h)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,8)
plt.show()

data_hv_s_rs['beam_power'] = data_hv_s_rs['HV_voltage'] * data_hv_s_rs['HV_current']
ref_counts_norm_beam_lead = ref_counts_norm_lead/data_hv_s_rs['beam_power']

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_beam_lead.index, y=ref_counts_norm_beam_lead, ax=ax)

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by \n dose and beam power \n [1/(30s) * 1/(muSv/h) * 1/(W)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,0.125)
plt.show()


ref_counts_norm_HV_lead = ref_counts_norm_lead/data_hv_s_rs['HV_voltage']

fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_HV_lead.index, y=ref_counts_norm_HV_lead, ax=ax)

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by \n dose and high voltage \n [1/(30s) * 1/(muSv/h) * 1/(kV)]')
ax.set_xlabel(f'Time, {DAY}')
plt.ylim(0,0.1)
plt.show()

## Plot both

In [ ]:
ref_counts_norm_HV_lead = ref_counts_norm_HV_lead.reset_index(drop=True)
ref_counts_norm_HV_no_lead = ref_counts_norm_HV_no_lead.reset_index(drop=True)
ref_counts_norm_lead = ref_counts_norm_lead.reset_index(drop=True)
ref_counts_norm_no_lead = ref_counts_norm_no_lead.reset_index(drop=True)
ref_counts_norm_beam_lead = ref_counts_norm_beam_lead.reset_index(drop=True)
ref_counts_norm_beam_no_lead = ref_counts_norm_beam_no_lead.reset_index(drop=True)


In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_HV_lead.index, y=ref_counts_norm_HV_lead, ax=ax, color='darkred', label="lead")
sns.lineplot(x=ref_counts_norm_HV_no_lead.index, y=ref_counts_norm_HV_no_lead, ax=ax, color='darkblue', label="no lead")

# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by \n dose and high voltage \n [1/(30s) * 1/(muSv/h) * 1/(kV)]')
ax.set_xlabel(f'30 sec readout interval')
plt.ylim(0,0.1)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.lineplot(x=ref_counts_norm_lead.index, y=ref_counts_norm_lead, ax=ax, color='darkred', label="lead")
sns.lineplot(x=ref_counts_norm_no_lead.index, y=ref_counts_norm_no_lead, ax=ax, color='darkblue', label="no lead")

# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by dose \n [1/(30s) * 1/(muSv/h)]')
ax.set_xlabel(f'30 sec readout interval')
plt.ylim(0,8)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.set()
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

sns.lineplot(x=ref_counts_norm_beam_lead.index, y=ref_counts_norm_beam_lead, ax=ax, color='darkred', label="lead")
sns.lineplot(x=ref_counts_norm_beam_no_lead.index, y=ref_counts_norm_beam_no_lead, ax=ax, color='darkblue', label="no lead")

# ax.set(yscale="log")
ax.set_ylabel('Counts normalized by \n dose and beam power \n [1/(30s) * 1/(muSv/h) * 1/(W)]')
ax.set_xlabel(f'30 sec readout interval')

plt.ylim(0,0.125)
plt.show()

# Check the emitting spot size detector